## Predicting FinTech Bootcamp Graduate Salaries
##### Project 2 - Group C (Andrew, Margee, Rachel, Jinhyeong)
##### Presentation Date: January 15, 2022

#### Data Preparation

In [118]:
# Imports
import json
import pandas as pd
import numpy as np
import warnings
import hvplot.pandas
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
warnings.filterwarnings("ignore")

In [119]:
# Create File Path
filepath = "C:/Users/piercerachel/Desktop/git/Project2_GroupC/"
filename = "monster_india_latest_jobs_free_dataset.json"

# Opening JSON file
f = open(filepath + filename)

# returns JSON object as a dictionary
data = json.load(f)

In [120]:
# Create data list
address_country = []
address_locality = []
address_region = []
company = []
description = []
industry = []
postal_code = []
posted_at = []
salary = []  # the target
salary_type = []
skills = []
title = []
url = []

# iterate through each element in the data list
for each in data:

    # append values to list, given each key
    address_country.append(each["address_country"])
    address_locality.append(each["address_locality"])
    address_region.append(each["address_region"])
    company.append(each["company"])
    description.append(each["description"])
    industry.append(each["industry"])
    postal_code.append(each["postal_code"])
    posted_at.append(each["posted_at"])
    salary.append(each["salary"])
    salary_type.append(each["salary_type"])
    skills.append(each["skills"]) 
    title.append(each["title"])
    url.append(each["url"])

In [121]:
# create df, using string header and list as rows
df = pd.DataFrame({"address_country": address_country,
                   "address_locality": address_locality,
                   "address_region": address_region,
                   "company": company,
                   "description": description,
                   "industry": industry,
                   "postal_code": postal_code,
                   "posted_at": posted_at,
                   "salary": salary,
                   "salary_type": salary_type,
                   "skills": skills,
                   "title": title,
                   "url": url,
                  })

df["description"] = df["description"].str.lower()

In [122]:
# Preview dataframe
df.head()

,address_country,address_locality,address_region,company,description,industry,postal_code,posted_at,salary,salary_type,skills,title,url
0,IN,India,India,Kotak Mahindra Bank Limited,job description : - . client engagement o...,Banking/Accounting/Financial Services,NA,2021-09-16,Not disclosed,P.A.,Service Relationship Manager-POS-BRANCH BANKIN...,Service Relationship Manager-POS-BRANCH BANKIN...,https://www.monsterindia.com/job/service-relat...
1,IN,Noida,Uttar Pradesh,ANI Calls India Private Limited,job description : . axway b2bi or axway gatew...,IT/Computers - Software,NA,2021-09-13,1600000,P.A.,Axway B2B EDI Integration consultant,Axway B2B EDI Integration consultant,https://www.monsterindia.com/job/axway-b2b-edi...
2,IN,Chennai,Tamil Nadu,PayPal,job description : job description fueled by a...,Banking/Accounting/Financial Services,NA,2021-08-18,Not disclosed,P.A.,Data Engineer/MTS 2,"MTS 2, Data Engineer",https://www.monsterindia.com/job/mts-2-data-en...
3,IN,Bengaluru / Bangalore,Karnataka,ANI Calls India Private Limited,job description : at least three years weblog...,IT/Computers - Software,NA,2021-08-26,1800000,P.A.,Senior Oracle Fusion Middleware developer,Senior Oracle Fusion Middleware developer,https://www.monsterindia.com/job/senior-oracle...
4,IN,India,India,ANI Calls India Private Limited,job description : • linux engineer with ksh s...,IT/Computers - Software,NA,2021-09-08,1400000,P.A.,Korn Shell (KSH) - Linux System Engineer,Korn Shell (KSH) - Linux System Engineer,https://www.monsterindia.com/job/korn-shell-ks...


In [123]:
# Clean data, drop unnecessary columns
df.drop(['address_country', 'address_locality', 'address_region', 'postal_code', 'salary_type', 'skills', 'url'], axis=1, inplace=True)

In [124]:
df.head()

,company,description,industry,posted_at,salary,title
0,Kotak Mahindra Bank Limited,job description : - . client engagement o...,Banking/Accounting/Financial Services,2021-09-16,Not disclosed,Service Relationship Manager-POS-BRANCH BANKIN...
1,ANI Calls India Private Limited,job description : . axway b2bi or axway gatew...,IT/Computers - Software,2021-09-13,1600000,Axway B2B EDI Integration consultant
2,PayPal,job description : job description fueled by a...,Banking/Accounting/Financial Services,2021-08-18,Not disclosed,"MTS 2, Data Engineer"
3,ANI Calls India Private Limited,job description : at least three years weblog...,IT/Computers - Software,2021-08-26,1800000,Senior Oracle Fusion Middleware developer
4,ANI Calls India Private Limited,job description : • linux engineer with ksh s...,IT/Computers - Software,2021-09-08,1400000,Korn Shell (KSH) - Linux System Engineer


In [125]:
# Drop any salaries that are Not disclosed
df = df[df["salary"] != "Not disclosed"]
df.head()

,company,description,industry,posted_at,salary,title
1,ANI Calls India Private Limited,job description : . axway b2bi or axway gatew...,IT/Computers - Software,2021-09-13,1600000,Axway B2B EDI Integration consultant
3,ANI Calls India Private Limited,job description : at least three years weblog...,IT/Computers - Software,2021-08-26,1800000,Senior Oracle Fusion Middleware developer
4,ANI Calls India Private Limited,job description : • linux engineer with ksh s...,IT/Computers - Software,2021-09-08,1400000,Korn Shell (KSH) - Linux System Engineer
5,ANI Calls India Private Limited,job description : . oracle ebs apps dba imple...,IT/Computers - Software,2021-09-16,1400000,Oracle Cloud Infrastructure
7,Live Connections,requirement mulesoft developer,Recruitment/Staffing/RPO,2021-08-30,2500000,Mulesoft Developer


In [126]:
# Check if there are any NAs/nulls
df.isnull().sum()

company        0
description    0
industry       0
posted_at      0
salary         0
title          0
dtype: int64

In [127]:
# Creating Keywords for Searching
keywords_sql = "sql|oracle|database|db|datawarehouse|datawarehousing"
keywords_python = "python|scripting"
keywords_api = "api"
keywords_algorithm = "algorithm|algorithmic"
keywords_aws = "aws|amazon web services"
keywords_forecast = "forecast|predict"
keywords_bigdata = "bigdata|big data"

In [128]:
# Searching for keyword skills and updating columns from 0s to 1s as applicable
df["sql"] = np.where(df["description"].str.contains(keywords_sql), 1, 0)
df["python"] = np.where(df["description"].str.contains(keywords_python), 1, 0)
df["api"] = np.where(df["description"].str.contains(keywords_api), 1, 0)
df["algorithm"] = np.where(df["description"].str.contains(keywords_algorithm), 1, 0)
df["aws"] = np.where(df["description"].str.contains(keywords_aws), 1, 0)
df["forecast"] = np.where(df["description"].str.contains(keywords_forecast), 1, 0)
df["bigdata"] = np.where(df["description"].str.contains(keywords_bigdata), 1, 0)

In [129]:
# Show updated dataframe
df.head()

,company,description,industry,posted_at,salary,title,sql,python,api,algorithm,aws,forecast,bigdata
1,ANI Calls India Private Limited,job description : . axway b2bi or axway gatew...,IT/Computers - Software,2021-09-13,1600000,Axway B2B EDI Integration consultant,0,0,0,0,0,0,0
3,ANI Calls India Private Limited,job description : at least three years weblog...,IT/Computers - Software,2021-08-26,1800000,Senior Oracle Fusion Middleware developer,1,0,0,0,0,0,0
4,ANI Calls India Private Limited,job description : • linux engineer with ksh s...,IT/Computers - Software,2021-09-08,1400000,Korn Shell (KSH) - Linux System Engineer,0,1,0,0,0,0,0
5,ANI Calls India Private Limited,job description : . oracle ebs apps dba imple...,IT/Computers - Software,2021-09-16,1400000,Oracle Cloud Infrastructure,1,0,0,0,1,0,0
7,Live Connections,requirement mulesoft developer,Recruitment/Staffing/RPO,2021-08-30,2500000,Mulesoft Developer,0,0,0,0,0,0,0


In [130]:
# Drop all rows where all keywords equal 0
df_updated = df[~(df[['sql','python','api','algorithm','aws','forecast','bigdata']] == 0).all(axis=1)]
df_updated.reset_index(drop=True, inplace=True)
df_updated.head()

,company,description,industry,posted_at,salary,title,sql,python,api,algorithm,aws,forecast,bigdata
0,ANI Calls India Private Limited,job description : at least three years weblog...,IT/Computers - Software,2021-08-26,1800000,Senior Oracle Fusion Middleware developer,1,0,0,0,0,0,0
1,ANI Calls India Private Limited,job description : • linux engineer with ksh s...,IT/Computers - Software,2021-09-08,1400000,Korn Shell (KSH) - Linux System Engineer,0,1,0,0,0,0,0
2,ANI Calls India Private Limited,job description : . oracle ebs apps dba imple...,IT/Computers - Software,2021-09-16,1400000,Oracle Cloud Infrastructure,1,0,0,0,1,0,0
3,ANI Calls India Private Limited,job description : . principal engineer cloud ...,IT/Computers - Software,2021-08-23,1400000,Infrastructure as a Service (IaaS),1,0,0,0,0,0,0
4,ANI Calls India Private Limited,job description : . extensive enterprise cons...,IT/Computers - Software,2021-08-30,1800000,SFDC Architect consultant,0,0,1,0,0,0,0


In [131]:
# Convert salary data type before currency conversion
df_updated['salary'] = df_updated['salary'].astype('int')
df_updated.dtypes

company        object
description    object
industry       object
posted_at      object
salary          int32
title          object
sql             int32
python          int32
api             int32
algorithm       int32
aws             int32
forecast        int32
bigdata         int32
dtype: object

In [132]:
# Convert Indian rupees to US dollars (conversion rate is as of 1/8/2022)
df_updated['salary'] = round(df_updated['salary']/74.28, 2)
df_updated.drop(columns=['company', 'description', 'industry', 'posted_at', 'title'], inplace=True)
df_updated.head()

,salary,sql,python,api,algorithm,aws,forecast,bigdata
0,24232.63,1,0,0,0,0,0,0
1,18847.60,0,1,0,0,0,0,0
2,18847.60,1,0,0,0,1,0,0
3,18847.60,1,0,0,0,0,0,0
4,24232.63,0,0,1,0,0,0,0


In [133]:
# Determine data count
df_updated.shape

(175, 8)

In [134]:
# Review new data set
df_updated.describe()

,salary,sql,python,api,algorithm,aws,forecast,bigdata
count,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,19180.628114,0.605714,0.262857,0.280000,0.057143,0.137143,0.051429,0.057143
std,7907.483036,0.490099,0.441449,0.450287,0.232781,0.344985,0.221504,0.232781
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16828.220000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,18847.600000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,21540.120000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,46445.880000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [135]:
# TUESDAY CLASS - MY EDITS BEGIN HERE

# Now we need to create a column at the end that totals up the number of 1's in each row
# This will help us determine if higher salaries correspond with more skills
# New dataframe will simply be two columns - salary and total_skillls

df_updated['total_skills'] = df_updated[['sql','python','api','algorithm','aws','forecast','bigdata']].sum(axis=1)

In [110]:
# Preview updated dataframe with total_skills column
df_updated

,salary,sql,python,api,algorithm,aws,forecast,bigdata,total_skills
0,24232.63,1,0,0,0,0,0,0,1
1,18847.60,0,1,0,0,0,0,0,1
2,18847.60,1,0,0,0,1,0,0,2
3,18847.60,1,0,0,0,0,0,0,1
4,24232.63,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
170,24232.63,0,1,0,0,0,0,0,1
171,24232.63,1,1,0,0,0,0,0,2
172,33656.44,1,0,0,0,0,0,0,1
173,201.94,0,0,1,0,0,0,0,1


In [136]:
# Now we drop the other columns
df_updated.drop(columns=['sql','python','api','algorithm','aws','forecast','bigdata'], inplace=True)

In [140]:
# Preview new dataframe
df_updated

,salary,total_skills
0,24232.63,1
1,18847.60,1
2,18847.60,2
3,18847.60,1
4,24232.63,1
...,...,...
170,24232.63,1
171,24232.63,2
172,33656.44,1
173,201.94,1


#### Training and Testing

In [299]:
# Next step is to train/test the data!
# salary = y, dummy columns (SQL, etc) are x, split data, train on first 70% of observations with number regression
# Run model on test, see if it performs
# Regression - Predict salary based on skills learned in class OR Classification - cutoff point for salary
# Optional Chatbot of what skills do you possess = salary range in rupees

In [141]:
# Set the random seed
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [142]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df_updated, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df_updated) - window - 1):
        features = df_updated.iloc[i:(i + window), feature_col_number]
        target = df_updated.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [159]:
# Predict Salaries beginning with window 10
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 1 is the 'total_skills' column - IS THIS RIGHT?  
# Column index 0 is the `salary` column
#THE OUTPUTS BELOW SEEM TO BE PREDICTING THE TOTAL SKILLS COLUMN, NOT SALARY
feature_column = 1
target_column = 0
X, y = window_data(df_updated, window_size, feature_column, target_column)

In [160]:
# Use 70% of the data for training and the remainder for testing

split = int(0.7 * len(X))

X_train = X[: split]
X_test = X[split:]

y_train = y[: split]
y_test = y[split:]
split

114

In [161]:
# Use the MinMaxScaler to scale data between 0 and 1.

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit the MinMaxScaler object with the features data X
scaler.fit(X_train)

# Scale the features training and testing sets
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Fit the MinMaxScaler object with the target data Y
scaler.fit(y_train)

# Scale the target training and testing sets
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [162]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Print some sample data after reshaping the datasets
print (f"X_train sample values:\n{X_train[:3]} \n")
print (f"X_test sample values:\n{X_test[:3]}")

X_train sample values:
[[[0.  ]
  [0.  ]
  [0.25]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.25]
  [0.  ]
  [0.  ]
  [0.25]]

 [[0.  ]
  [0.25]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.25]
  [0.  ]
  [0.  ]
  [0.25]
  [0.  ]]

 [[0.25]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.25]
  [0.  ]
  [0.  ]
  [0.25]
  [0.  ]
  [0.  ]]] 

X_test sample values:
[[[0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.25]]

 [[0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.25]
  [0.  ]]

 [[0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.  ]
  [0.25]
  [0.  ]
  [0.5 ]]]


### Build and Train the Model

In [147]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

# Define the LSTM RNN model.
model = Sequential()

# Initial model setup
number_units = 30
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))

In [148]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [149]:
# Summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 30)            3840      
                                                                 
 dropout (Dropout)           (None, 10, 30)            0         
                                                                 
 lstm_1 (LSTM)               (None, 10, 30)            7320      
                                                                 
 dropout_1 (Dropout)         (None, 10, 30)            0         
                                                                 
 lstm_2 (LSTM)               (None, 30)                7320      
                                                                 
 dropout_2 (Dropout)         (None, 30)                0         
                                                                 
 dense (Dense)               (None, 1)                 3

In [150]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiment with the batch size, but a smaller batch size is recommended

model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=90, verbose=1)

Epoch 1/10
2/2 [==============================] - 5s 0s/step - loss: 0.0559
Epoch 2/10
2/2 [==============================] - 0s 14ms/step - loss: 0.0510
Epoch 3/10
2/2 [==============================] - 0s 17ms/step - loss: 0.0471
Epoch 4/10
2/2 [==============================] - 0s 13ms/step - loss: 0.0451
Epoch 5/10
2/2 [==============================] - 0s 21ms/step - loss: 0.0436
Epoch 6/10
2/2 [==============================] - 0s 15ms/step - loss: 0.0442
Epoch 7/10
2/2 [==============================] - 0s 18ms/step - loss: 0.0444
Epoch 8/10
2/2 [==============================] - 0s 16ms/step - loss: 0.0432
Epoch 9/10
2/2 [==============================] - 0s 14ms/step - loss: 0.0442
Epoch 10/10
2/2 [==============================] - 0s 24ms/step - loss: 0.0446


#### Model Performance

In [151]:
# Evaluate the model
model.evaluate(X_test, y_test, verbose=0)

0.03427726402878761

In [152]:
# Make some predictions
predicted = model.predict(X_test)

In [153]:
# Recover the original prices instead of the scaled version
predicted_salary = scaler.inverse_transform(predicted)
real_salary = scaler.inverse_transform(y_test.reshape(-1, 1))

In [154]:
# Create a DataFrame of Real and Predicted values
salaries = pd.DataFrame({
    "Actual": real_salary.ravel(),
    "Predicted": predicted_salary.ravel()
}, index = df_updated.index[-len(real_salary): ]) 

# Show the DataFrame's head
salaries.head()

,Actual,Predicted
125,1.0,1.316291
126,3.0,1.319330
127,2.0,1.326660
128,1.0,1.337467
129,3.0,1.348580


In [ ]:
# Plot the real vs predicted values as a line chart
stocks.plot(title="Actual Vs. Predicted Salaries")

### DASHBOARD

#### We created a dashboard for the presentation. The dashboard was saved as an html.

##### First Step: Create Markdowns for Overview Tabs

In [ ]:
markdown_titlepage = """

# Predicting FinTech Bootcamp Graduate Salaries  
  
Presentation Date: January 15, 2022 
    
Prepared by Andrew, Rachel, Margee, and Jinhyeong  
  
![image](./images/XXXX.png)

"""

In [ ]:
markdown_intro = """
# Project Objective  
  
Our project is to research and learn what salaries are possible with skills obtained in this FinTech Bootcamp.  
We will be using the Monster India API which includes salaries in rupees since the US-based API required a  
significant fee to use.  We will use the API to retrieve data regarding job descriptions, skills, and salaries  
to determine the following:  
- What jobs require skills from this bootcamp?  
- What are the potential jobs and salaries for graduates with these skills?  
- Can we predict an accurate salary range based on these skills?  

  
![image](./images/XXXX.png)
"""

In [ ]:
markdown_background1 = """
# What Skills Have We Gained as FinTech Bootcamp Students?
  
As part of the U of MN FinTech Bootcamp, we have obtained the necessary skills to automate  
and improve financial services using cutting-edge technology. Skills gained inclde the following:
  
![image](./images/Skills.png)

"""

In [ ]:
markdown_background2 = """
# Background of Key Skills Used During This Project
  
SQL  
Python  
API  
Algorithm  
AWS  
Forecast  
Big data  
  
  
![image](./XX.png)

"""

In [ ]:
markdown_end = """

# Conclusion  
  
Based on our analysis, FinTech Bootcamp graduates can potentially earn a salary between X and X.
  

  
  
![image](./images/XX.png)
"""

In [ ]:
dataphases = """

# Data Phases  
  
Our process consisted of a data exploration, cleanup, and analysis phases.  
  
>- Exploration: Google seaches, API searches, Monster India API, FinTech Bootcamp Curriculum  
>- Cleanup: Searching through variables in Spyder, selecting key data, dropping unnecessary information, narrowing down jobs
>- Analysis: Looking through the data to find jobs that include at least one key skill
>- Training/Testing:
>- Create Model to Predict Salary based on Skills:
>- Fun ChatBot:
  
"""

##### Second Step: Fix any dashboard issues

In [ ]:
#Some of our plots did not translate well in the dashboard, so we converted them to pngs and used the image
plot_1 ="""
![image](./images/plot_1.png)
"""

##### Third Step: Create the Dashboard

In [ ]:
#Create Dashboard
pn.Tabs(
    ("XX",markdown_titlepage),
    ("XX",markdown_background1),
    ("XX",markdown_background2),
    ("XX",markdown_background3),
    ("XX",markdown_background4),
    ("XX",visualoverview),
    ("XX",plot_1),
    ("XX",plot_2),
    ("XX",plot_3),
    ("XX",plot_4),
    ("XX",plot_5),
    ("XX",plot_6),
    ("XX",plot_7),
    ("XX",markdown_end)
       )

##### Final Step: Save to html for presentation

In [ ]:
#Create an html of the dashboard for presentation purposes once finalized
pn.Tabs(
    ("XX",markdown_titlepage),
    ("XX",markdown_background1),
    ("XX",markdown_background2),
    ("XX",markdown_background3),
    ("XX",markdown_background4),
    ("XX",visualoverview),
    ("XX",plot_1),
    ("XX",plot_2),
    ("XX",plot_3),
    ("XX",plot_4),
    ("XX",plot_5),
    ("XX",plot_6),
    ("XX",plot_7),
    ("XX",markdown_end)
       ).save("Dashboard_Presentation.html", embed=True)